## Retrieve data from brokerages and save to files.
last modified: 1/12/2018

In [5]:
%load_ext autoreload
%autoreload 1
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from time import sleep

In [7]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('.'), '..', 'lib'))

from tdameritrade import tdameritrade_api
from interactivebrokers import ib_api as IB_API
from strategies import orders

In [9]:
ib_api = IB_API.main(host="192.168.1.2", port=7496)

serverVersion:142 connectionTime:b'20190828 20:53:29 EST'


ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds


-1
-1
-1
-1
-1


In [10]:
td_api = tdameritrade_api.TDAmeritradeAPI.create_api_from_account_file(filename="../private_data/information.json")

HTTPError: HTTP Error 400: Bad Request

### Set constants

In [13]:
with open("../private_data/information.json") as file_obj:
    information = json.load(file_obj)

ACCOUNT_IRA = information["IB_IRA"]
ACCOUNT_IND = information["IB_IND"]

In [14]:
DATA = "/home/brent/.jupyter/data/data.hdf"

### Get saved data

In [15]:
with pd.HDFStore(DATA, mode="r") as hdf_store:
    bars = hdf_store["bars"]
    fundamentals = hdf_store["fundamentals"]
    quotes = hdf_store["quotes"]
    positions = hdf_store["positions"]
    universe = bars.columns.get_level_values(0).unique()

### Separate data into logical variables

In [16]:
indices = ["SPY", "QQQ", "DIA", "IWM"]
snp = list(set(bars.columns.get_level_values(0).unique()) - set(indices))

In [17]:
indices = bars[indices]
snp = bars[snp]

### Get positions

In [18]:
positions = ib_api.get_positions()
positions['account'] = positions.index
positions = positions.set_index('symbol')
with pd.HDFStore(DATA, mode="a") as hdf_store:
    hdf_store.put("positions", positions)

### Get price history

#### Get daily history

In [23]:
start_date = datetime(2019, 8, 24)
end_date = datetime(2019, 8, 29)

In [24]:
temp = ib_api.get_price_history(universe[0], start_date=start_date, end_date=end_date, bar_size="1 day", rth=True)

In [27]:
symbols = sorted(list(set(universe) - set(temp.dropna(axis=1).columns.get_level_values(0).unique())))
for symbol in symbols:
    try:
        temp = temp.join(ib_api.get_price_history(symbol, start_date=start_date, end_date=end_date))
        sleep(0.4)
    except:
        print(symbol)
        pass
dropped = set(universe) - set(temp.columns.get_level_values(0))
if len(dropped) != 0:
    print(dropped)

ERROR 1717 399 Order Message:
BUY 6 ROKU NASDAQ.NMS
ERROR 1718 399 Order Message:
SELL 6 ROKU NASDAQ.NMS
ERROR 1719 399 Order Message:
SELL 6 ROKU NASDAQ.NMS


1717
1718
1719


ERROR 1718 202 Order Canceled - reason:
ERROR 1719 202 Order Canceled - reason:


1718
1719


ERROR 1717 202 Order Canceled - reason:


1717


In [32]:
#bars = bars.drop(list(dropped), axis=1)
bars = bars.append(temp)

In [33]:
with pd.HDFStore(DATA, mode="a") as hdf_store:
    hdf_store.put("daily_bars", bars)

#### Get hour data

In [100]:
end_date = datetime.today()
end_date = datetime(end_date.year, end_date.month, end_date.day, 4, 0, 0)
start_date = end_date - timedelta(days=5)
symbols = "QQQ, VNQ, XLF, XLV, XLY, XLP, GDX, XLU, XLE, XLI".split(", ")
data = ib_api.get_price_history(symbols, start_date=start_date, end_date=end_date, bar_size="30 mins", rth=False)

In [98]:
with pd.HDFStore(DATA, mode="a") as hdf_store:
    hdf_store.put("hourly_bars", data)

#### Get minute data

In [5]:
end_date = datetime(2019, 8, 1)
end_date = datetime(end_date.year, end_date.month, end_date.day, 4, 0, 0)
start_date = end_date - timedelta(days=1)
symbols = "QQQ, VNQ, XLF, XLV, XLY, XLP, GDX, XLU, XLE, XLI".split(", ")
data = ib_api.get_price_history(symbols, start_date=start_date, end_date=end_date, bar_size="1 min", rth=False)

> /home/brent/.jupyter/lib/interactivebrokers/ib_api.py(594)get_price_history()
-> duration = end_date - start_date
(Pdb) c


In [11]:
with pd.HDFStore(DATA, mode="a") as hdf_store:
    hdf_store.put("minute_bars", data)

ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.


-1


ERROR -1 2103 Market data farm connection is broken:usfarm
ERROR -1 2103 Market data farm connection is broken:secdefnj


-1
-1


ERROR -1 1102 Connectivity between IB and Trader Workstation has been restored - data maintained.
ERROR -1 2104 Market data farm connection is OK:secdefnj


...

-1


ERROR -1 2105 HMDS data farm connection is broken:euhmds


-1


ERROR -1 2106 HMDS data farm connection is OK:euhmds


-1


### Get fundamentals

In [50]:
tickers = snp.columns.get_level_values(0).unique()
temp = td_api.get_fundamental(tickers[:10])
for i in range(10, 490, 10):
    temp = temp.append(td_api.get_fundamental(tickers[i: i+10]))
    sleep(10)
fundamentals = temp.append(td_api.get_fundamental(tickers[490:]))

INFO:tdameritrade.tdameritrade_api:URL: https://api.tdameritrade.com/v1/instruments?symbol=JCI%2CEQIX%2CCMI%2CSYF%2CEQR%2CPAYX%2CVRSN%2CTIF%2CETR%2CXRX&projection=fundamental
INFO:tdameritrade.tdameritrade_api:URL: https://api.tdameritrade.com/v1/instruments?symbol=MSFT%2CLYB%2CTRIP%2CYUM%2CFIS%2CDGX%2CABC%2CRF%2CLRCX%2CINFO&projection=fundamental
INFO:tdameritrade.tdameritrade_api:URL: https://api.tdameritrade.com/v1/instruments?symbol=SCHW%2CFLT%2CFTV%2CKLAC%2CHPQ%2CZBH%2CLKQ%2CEXR%2CMKC%2CLLL&projection=fundamental
INFO:tdameritrade.tdameritrade_api:URL: https://api.tdameritrade.com/v1/instruments?symbol=LB%2CBWA%2CRCL%2CWYNN%2CIR%2CJPM%2CMRO%2CGD%2CEW%2CAAPL&projection=fundamental
INFO:tdameritrade.tdameritrade_api:URL: https://api.tdameritrade.com/v1/instruments?symbol=ADS%2CUHS%2CNSC%2CTJX%2CCTXS%2CPRU%2CD%2CDUK%2CDIS%2CDHR&projection=fundamental
INFO:tdameritrade.tdameritrade_api:URL: https://api.tdameritrade.com/v1/instruments?symbol=KEY%2CLEG%2CAOS%2CC%2CNRG%2CADI%2CSYY%2CISRG

INFO:tdameritrade.tdameritrade_api:URL: https://api.tdameritrade.com/v1/instruments?symbol=STZ%2CNDAQ%2CCSX%2CADP%2CBHGE%2CFDX%2CDXC%2CSJM%2CNOV%2CCINF&projection=fundamental
INFO:tdameritrade.tdameritrade_api:URL: https://api.tdameritrade.com/v1/instruments?symbol=SEE%2CAIZ%2CLW%2CQCOM%2CXRAY%2CMPC%2CFOX%2CMXIM&projection=fundamental


In [53]:
cols = "beta, bookValuePerShare, currentRatio, divGrowthRate3Year, dividendAmount, dividendPayAmount, dividendYield, epsChange, "+\
       "epsChangePercentTTM, epsChangeYear, epsTTM, high52, low52, pbRatio, pcfRatio, peRatio, pegRatio, prRatio, quickRatio"
cols = cols.split(", ")
for col in cols:
    fundamentals[col] = fundamentals[col].astype(float)

cols = "grossMarginMRQ, grossMarginTTM, interestCoverage, ltDebtToEquity, marketCap, marketCapFloat, netProfitMarginMRQ, netProfitMarginTTM, "+\
       "operatingMarginMRQ, operatingMarginTTM, returnOnAssets, returnOnEquity, returnOnInvestment, revChangeIn, revChangeTTM, revChangeYear, "+\
       "sharesOutstanding, shortIntDayToCover, shortIntToFloat, totalDebtToCapital, totalDebtToEquity, vol10DayAvg, vol1DayAvg, vol3MonthAvg"
cols = cols.split(", ")
for col in cols:
    fundamentals[col] = fundamentals[col].astype(int)

In [55]:
with pd.HDFStore(DATA, mode="a") as hdf_store:
    hdf_store.put("fundamentals", fundamentals)

/home/brent/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2903: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['dividendDate', 'dividendPayDate', 'symbol']]

  if self.run_code(code, result):


### RPC for Quotes

In [10]:
#temp = td_api.get_quotes(universe[0])
temp = ib_api.get_quotes(universe[0])

In [16]:
symbols = sorted(list(set(universe) - set(temp.index)))
for symbol in symbols:
    #temp = temp.append(td_api.get_quotes(symbol))
    temp = temp.append(ib_api.get_quotes(symbol))

In [17]:
quotes = quotes.append(temp)

In [18]:
with pd.HDFStore("data.hdf", mode="a") as hdf_store:
    hdf_store.put("quotes", quotes)